# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2019
topic: str = "BIOLOGÍA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [4]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. El potencial de reposo de las neuronas es p...
3,mo a -70 mV porque:
4,
5,1. La membrana es muy permeable al Na+.
6,2. Los canales de Ca2+ están abiertos.
7,3. La permeabilidad de su membrana al K+ es
8,mayor que para otros iones.
9,4. Los canales de K+ están cerrados.


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3,38,4,75,3.0,112,4,149,4.0
1,2,4,39,1,76,3.0,113,4,150,4.0
2,3,3,40,4,77,NaN,114,2,151,4.0
3,4,1,41,3,78,NaN,115,3,152,4.0
4,5,1,42,2,79,4.0,116,3,153,3.0
5,6,1,43,1,80,1.0,117,4,154,NaN
6,7,3,44,2,81,1.0,118,2,155,3.0
7,8,1,45,3,82,4.0,119,1,156,2.0
8,9,4,46,1,83,2.0,120,4,157,3.0
9,10,4,47,4,84,1.0,121,4,158,2.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [6]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("FSE BIOLOGÍA")]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [7]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [8]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [9]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [10]:
raw_exam_df.head(10)

0    1. El potencial de reposo de las neuronas es p...
1                                  mo a -70 mV porque:
2              1. La membrana es muy permeable al Na+.
3               2. Los canales de Ca2+ están abiertos.
4          3. La permeabilidad de su membrana al K+ es
5                          mayor que para otros iones.
6                 4. Los canales de K+ están cerrados.
7    2. La liberación de neurotransmisor en las sin...
8                                            químicas:
9     1. Es independiente de la concentración de Ca2+.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [11]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [12]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. El potencial de reposo de las neuronas es p...
1              1. La membrana es muy permeable al Na+.
2               2. Los canales de Ca2+ están abiertos.
3    3. La permeabilidad de su membrana al K+ es ma...
4                 4. Los canales de K+ están cerrados.
5    2. La liberación de neurotransmisor en las sin...
6     1. Es independiente de la concentración de Ca2+.
7    2. Se evoca por la interacción del Na+ con la ...
8                 3. Se produce por transporte activo.
9                     4. Se produce de forma cuántica.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [13]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 2
Number of rows not ending with '.', ':' or '?' = 2
[744, 749]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [14]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [15]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [16]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    # rows_fix: list[int] = []
    # exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [17]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [18]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [19]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [20]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [21]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [22]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. El potencial de reposo de las neuronas es p...,1. La membrana es muy permeable al Na+.,2. Los canales de Ca2+ están abiertos.,3. La permeabilidad de su membrana al K+ es ma...,4. Los canales de K+ están cerrados.
1,2. La liberación de neurotransmisor en las sin...,1. Es independiente de la concentración de Ca2+.,2. Se evoca por la interacción del Na+ con la ...,3. Se produce por transporte activo.,4. Se produce de forma cuántica.
2,3. Los receptores para moléculas odoríferas:,1. Son de tipo ionotrópico.,2. Tienen actividad tirosina cinasa.,3. Están acoplados a proteínas G.,4. Se localizan en el citosol.
3,4. La corteza motora primaria se localiza en e...,1. Frontal.,2. Parietal.,3. Temporal.,4. Occipital.
4,5. El reflejo flexor:,1. Requiere inervación recíproca.,2. Es monosináptico.,3. Se evoca por estiramiento del músculo.,4. Se origina en el órgano tendinoso de Golgi.
5,"6. El electroencefalograma de un adulto sano, ...",1. Alfa.,2. Beta.,3. Gamma.,4. Delta.
6,7. En un electrocardiograma estándar:,1. La onda P es sincrónica con la contracción ...,2. El triángulo de Einthoven se dispone en el ...,3. La derivación I es un registro bipolar de l...,4. La derivación aVR es precordial.
7,8. El flujo coronario izquierdo disminuye por:,1. La sístole ventricular.,2. Adenosina.,3. Hipoxemia miocárdica moderada.,4. Óxido nítrico.
8,9. El fibrinógeno pasa a fibrina por acción de:,1. Factor de von Willebrand.,2. Prostaciclina.,3. Trombopoyetina.,4. Trombina.
9,10. El sistema nervioso simpático actúa sobre ...,1. Colinérgicos tipo nicotínicos.,2. Colinérgicos tipo muscarínicos.,3. Adrenérgicos tipo α.,4. Adrenérgicos tipo β.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [23]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [24]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [25]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [26]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,4.0
2,3,3.0
3,4,1.0
4,5,1.0
5,6,1.0
6,7,3.0
7,8,1.0
8,9,4.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [27]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. El potencial de reposo de las neuronas es p...,1. La membrana es muy permeable al Na+.,2. Los canales de Ca2+ están abiertos.,3. La permeabilidad de su membrana al K+ es ma...,4. Los canales de K+ están cerrados.,3.0
1,2. La liberación de neurotransmisor en las sin...,1. Es independiente de la concentración de Ca2+.,2. Se evoca por la interacción del Na+ con la ...,3. Se produce por transporte activo.,4. Se produce de forma cuántica.,4.0
2,3. Los receptores para moléculas odoríferas:,1. Son de tipo ionotrópico.,2. Tienen actividad tirosina cinasa.,3. Están acoplados a proteínas G.,4. Se localizan en el citosol.,3.0
3,4. La corteza motora primaria se localiza en e...,1. Frontal.,2. Parietal.,3. Temporal.,4. Occipital.,1.0
4,5. El reflejo flexor:,1. Requiere inervación recíproca.,2. Es monosináptico.,3. Se evoca por estiramiento del músculo.,4. Se origina en el órgano tendinoso de Golgi.,1.0
5,"6. El electroencefalograma de un adulto sano, ...",1. Alfa.,2. Beta.,3. Gamma.,4. Delta.,1.0
6,7. En un electrocardiograma estándar:,1. La onda P es sincrónica con la contracción ...,2. El triángulo de Einthoven se dispone en el ...,3. La derivación I es un registro bipolar de l...,4. La derivación aVR es precordial.,3.0
7,8. El flujo coronario izquierdo disminuye por:,1. La sístole ventricular.,2. Adenosina.,3. Hipoxemia miocárdica moderada.,4. Óxido nítrico.,1.0
8,9. El fibrinógeno pasa a fibrina por acción de:,1. Factor de von Willebrand.,2. Prostaciclina.,3. Trombopoyetina.,4. Trombina.,4.0
9,10. El sistema nervioso simpático actúa sobre ...,1. Colinérgicos tipo nicotínicos.,2. Colinérgicos tipo muscarínicos.,3. Adrenérgicos tipo α.,4. Adrenérgicos tipo β.,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [28]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [29]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [30]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [31]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. El potencial de reposo de las neuronas es p...,1. La membrana es muy permeable al Na+.,2. Los canales de Ca2+ están abiertos.,3. La permeabilidad de su membrana al K+ es ma...,4. Los canales de K+ están cerrados.,3,2019
1,2. La liberación de neurotransmisor en las sin...,1. Es independiente de la concentración de Ca2+.,2. Se evoca por la interacción del Na+ con la ...,3. Se produce por transporte activo.,4. Se produce de forma cuántica.,4,2019
2,3. Los receptores para moléculas odoríferas:,1. Son de tipo ionotrópico.,2. Tienen actividad tirosina cinasa.,3. Están acoplados a proteínas G.,4. Se localizan en el citosol.,3,2019
3,4. La corteza motora primaria se localiza en e...,1. Frontal.,2. Parietal.,3. Temporal.,4. Occipital.,1,2019
4,5. El reflejo flexor:,1. Requiere inervación recíproca.,2. Es monosináptico.,3. Se evoca por estiramiento del músculo.,4. Se origina en el órgano tendinoso de Golgi.,1,2019
5,"6. El electroencefalograma de un adulto sano, ...",1. Alfa.,2. Beta.,3. Gamma.,4. Delta.,1,2019
6,7. En un electrocardiograma estándar:,1. La onda P es sincrónica con la contracción ...,2. El triángulo de Einthoven se dispone en el ...,3. La derivación I es un registro bipolar de l...,4. La derivación aVR es precordial.,3,2019
7,8. El flujo coronario izquierdo disminuye por:,1. La sístole ventricular.,2. Adenosina.,3. Hipoxemia miocárdica moderada.,4. Óxido nítrico.,1,2019
8,9. El fibrinógeno pasa a fibrina por acción de:,1. Factor de von Willebrand.,2. Prostaciclina.,3. Trombopoyetina.,4. Trombina.,4,2019
9,10. El sistema nervioso simpático actúa sobre ...,1. Colinérgicos tipo nicotínicos.,2. Colinérgicos tipo muscarínicos.,3. Adrenérgicos tipo α.,4. Adrenérgicos tipo β.,4,2019


# Data Loading

## Saving in csv file

The `processed_exam_df` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [32]:
processed_exam_df.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [33]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [34]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [35]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()